爬取链家网站上武汉市二手房销售数据

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os

In [2]:
data_list=[]           # 用于存储一个页面下的房产的相关数据

In [3]:
def get_one_page(url):                    
    # 定义函数：打开网页，并该爬取网页的源代码
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38'}
    req = requests.get(url,headers=headers)
    req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text,'lxml')
    return soup

In [4]:
def save_data(data_list):
    data=pd.DataFrame(data_list)
    # 定义函数：用于将所爬取到的数据存储为csv文件
    if not os.path.exists("./data/武汉二手房.csv"):          # 若文件不存在，则创建新文件，此时要有表头
        data.to_csv("./data/武汉二手房.csv",encoding='ANSI',mode='a+',index=False)
    else:                                                    # 若文件已经存在，则将数据追加到该文件中，此时不加表头
        data.to_csv("./data/武汉二手房.csv",encoding='ANSI',mode='a+',index=False,header=False)

In [5]:
def parse_one_page(soup):                   
    # 定义函数：提取网页源代码中的有效信息
    try:       # 如果能够成功解析数据，就将数据存储到文件中
        items_list=soup.select('div[class="property-content"]')     # 返回一个列表，列表中的元素是每一个房子所对应的标签
        # 解析并存储每个房产中所要提取的特征
        for i in range(len(items_list)):
            # 解析
            item=items_list[i]
            title=item.h3.string           # 标题
            stru=item.select('p[class="property-content-info-text property-content-info-attribute"] span')    # 室厅卫
            num_s=stru[0].string           # 室数
            num_t=stru[2].string           # 厅数
            num_w=stru[4].string           # 卫数
            info=item.select('p[class="property-content-info-text"]')            
            pattern=re.compile("\S.*")
            surface=pattern.search(info[0].string).group()            # 面积
            direction=pattern.search(info[1].string).group()          # 朝向
            neighbor=item.select('p[class="property-content-info-comm-name"]')[0].string               # 小区
            location1=item.select('p[class="property-content-info-comm-address"] span')[0].string      # 行政区
            location2=item.select('p[class="property-content-info-comm-address"] span')[1].string      # 所属街道
            location3=item.select('p[class="property-content-info-comm-address"] span')[2].string      # 具体门牌号
            total_price=item.select('p[class="property-price-total"] span')[0].string+item.select('p[class="property-price-total"] span')[1].string    # 房屋总价
            average_price=item.select('p[class="property-price-average"]')[0].string      # 房屋单价
            # 注意到有些房产中没有楼层数和建造时间的相关信息，所以在此需要进行异常处理
            try:
                floors=pattern.search(info[2].string).group()             # 楼层数
                year=pattern.search(info[3].string).group()               # 建造时间
            except Exception as ex:
                floors=None
                year=None
            # 将零散数据整合到字典中
            data_dict={}
            data_dict["标题"]=title
            data_dict["室数"]=num_s
            data_dict["厅数"]=num_t
            data_dict["卫数"]=num_w
            data_dict["面积"]=surface
            data_dict["朝向"]=direction
            data_dict["楼层数"]=floors
            data_dict["建造时间"]=year
            data_dict["小区"]=neighbor
            data_dict["行政区"]=location1
            data_dict["街道"]=location2
            data_dict["门牌号"]=location3
            data_dict["单价"]=average_price
            data_dict["总价"]=total_price
            # 将该房产的相关数据构成的字典，追加到data_list列表中
            data_list.append(data_dict)
            pass
        save_data(data_list)
        print('已经成功爬取并保存')
    except Exception as ex:      # 如果解析出错，则打印错误信息
        print('出现错误，错误信息为：',ex)
    pass

In [6]:
def RunSpider(offset):                        # 定义执行函数
    url="https://wuhan.anjuke.com/sale/p{}/".format(offset)
    # 开始发起请求
    resultHtml=get_one_page(url)              # 获取该网页上的源代码信息
    # 解析并存储数据
    result=parse_one_page(resultHtml)         # 解析该页面数据，解析成功则保存，解析失败则打印错误信息
    pass

In [7]:
for i in range(1,31):         
    print(f'第{i}页数据的爬取情况：')
    RunSpider(i)
    # 添加一个延时的等待 防止速度过快被反爬
    time.sleep(3)

第1页数据的爬取情况：
已经成功爬取并保存
第2页数据的爬取情况：
已经成功爬取并保存
第3页数据的爬取情况：
已经成功爬取并保存
第4页数据的爬取情况：
已经成功爬取并保存
第5页数据的爬取情况：
已经成功爬取并保存
第6页数据的爬取情况：
已经成功爬取并保存
第7页数据的爬取情况：
已经成功爬取并保存
第8页数据的爬取情况：
已经成功爬取并保存
第9页数据的爬取情况：
已经成功爬取并保存
第10页数据的爬取情况：
已经成功爬取并保存
第11页数据的爬取情况：
已经成功爬取并保存
第12页数据的爬取情况：
已经成功爬取并保存
第13页数据的爬取情况：
已经成功爬取并保存
第14页数据的爬取情况：
已经成功爬取并保存
第15页数据的爬取情况：
已经成功爬取并保存
第16页数据的爬取情况：
已经成功爬取并保存
第17页数据的爬取情况：
已经成功爬取并保存
第18页数据的爬取情况：
已经成功爬取并保存
第19页数据的爬取情况：
已经成功爬取并保存
第20页数据的爬取情况：
已经成功爬取并保存
第21页数据的爬取情况：
已经成功爬取并保存
第22页数据的爬取情况：
已经成功爬取并保存
第23页数据的爬取情况：
已经成功爬取并保存
第24页数据的爬取情况：
已经成功爬取并保存
第25页数据的爬取情况：
已经成功爬取并保存
第26页数据的爬取情况：
已经成功爬取并保存
第27页数据的爬取情况：
已经成功爬取并保存
第28页数据的爬取情况：
已经成功爬取并保存
第29页数据的爬取情况：
已经成功爬取并保存
第30页数据的爬取情况：
已经成功爬取并保存
